### Some calculations prepared for the manuscript

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

####  Porosity distribution calculations

In [6]:
def porosity(z, z_swi, phi_swi, phi_inf):
    """Calculates porosity on depth z according to depth at SWI (z_swi),
                  porosity at SWI (phi_swi),
              and porosity at 'infinite depth' layer )phi_inf"""
    k_phi = 0.04
    k = -(z-z_swi)/k_phi
    return (phi_inf+(phi_swi-phi_inf)*np.exp(k))

In [7]:
porosity(2.505, 2.5, 0.75, 0.25)

0.691248451292299

In [8]:
z = np.linspace(2.505, 2.595, 10)
phi_inf_array = np.arange(0.05, 0.36, 0.01)
phi_swi_array = np.arange(0.45, 0.86, 0.01)

which combinations of phi_swi and phi_inf match appoximately 0.43:

In [9]:
for phi_swi in phi_swi_array:
    for phi_inf in phi_inf_array:
        if (0.432 < sum(porosity(z, 2.5, phi_swi, phi_inf))/10 < 0.434):
            print(phi_swi, phi_inf)

0.6100000000000001 0.33
0.6300000000000001 0.32000000000000006
0.6800000000000002 0.29000000000000004
0.7000000000000002 0.28
0.7300000000000002 0.26000000000000006
0.7500000000000002 0.25000000000000006
0.8000000000000003 0.22000000000000003
0.8200000000000003 0.21000000000000002


#### Alkalinity fluxes compounds

In [10]:
ds = xr.open_dataset('data/low_sulfate_reduction_rate/5_po75-25_di10e-9/water.nc')
alkflux_df = ds['B_C_Alk   _flux'].to_dataframe()
nh4flux_df = ds['B_NUT_NH4 _flux'].to_dataframe()
no2flux_df = ds['B_NUT_NO2 _flux'].to_dataframe()
no3flux_df = ds['B_NUT_NO3 _flux'].to_dataframe()
po4flux_df = ds['B_NUT_PO4 _flux'].to_dataframe()
so4flux_df = ds['B_S_SO4   _flux'].to_dataframe()
alkflux_bottom = alkflux_df.groupby('z_faces').get_group(2.5)
nh4flux_bottom = nh4flux_df.groupby('z_faces').get_group(2.5)
no2flux_bottom = no2flux_df.groupby('z_faces').get_group(2.5)
no3flux_bottom = no3flux_df.groupby('z_faces').get_group(2.5)
po4flux_bottom = po4flux_df.groupby('z_faces').get_group(2.5)
so4flux_bottom = so4flux_df.groupby('z_faces').get_group(2.5)

In [11]:
year = (('2011-01-01','2011-01-31'), ('2011-02-01','2011-02-28'), ('2011-03-01','2011-03-31'), ('2011-04-01','2011-04-30'), 
        ('2011-05-01','2011-05-31'), ('2011-06-01','2011-06-30'), ('2011-07-01','2011-07-31'), ('2011-08-01','2011-08-31'),
        ('2011-09-01','2011-09-30'), ('2011-10-01','2011-10-31'), ('2011-11-01','2011-11-30'), ('2011-12-01','2011-12-31'))

In [12]:
year_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [13]:
alk_year = []
nh4_year = []
no2_year = []
no3_year = []
po4_year = []
so4_year = []
for month in year:
    alk_month = alkflux_bottom.loc[month[0]:month[1]]
    nh4_month = nh4flux_bottom.loc[month[0]:month[1]]
    no2_month = no2flux_bottom.loc[month[0]:month[1]]
    no3_month = no3flux_bottom.loc[month[0]:month[1]]
    po4_month = po4flux_bottom.loc[month[0]:month[1]]
    so4_month = so4flux_bottom.loc[month[0]:month[1]]
    alk_year.append(alk_month['B_C_Alk   _flux'].mean())
    nh4_year.append(nh4_month['B_NUT_NH4 _flux'].mean())
    no2_year.append(no2_month['B_NUT_NO2 _flux'].mean())
    no3_year.append(no3_month['B_NUT_NO3 _flux'].mean())
    po4_year.append(po4_month['B_NUT_PO4 _flux'].mean())
    so4_year.append(so4_month['B_S_SO4   _flux'].mean())

In [14]:
alk = np.array(alk_year)
nh4 = np.array(nh4_year)
no2 = np.array(no2_year)
no3 = np.array(no3_year)
po4 = np.array(po4_year)
so4 = np.array(so4_year)

In [15]:
all_components = nh4-no2-no3-po4-2*so4

In [16]:
po4_quota = -po4/all_components
po4_quota

array([-0.01794102, -0.02222979, -0.02716499, -0.02478754, -0.01680583,
       -0.01177871, -0.01038317, -0.01019516, -0.01049379, -0.01108218,
       -0.01119354, -0.01356398])

In [17]:
nh4_quota = nh4/all_components
nh4_quota

array([0.20131813, 0.25774758, 0.2866659 , 0.24793261, 0.16267455,
       0.13330767, 0.1198205 , 0.11347282, 0.10583136, 0.09334922,
       0.10839223, 0.14702111])

In [18]:
no2_quota = -no2/all_components
no2_quota

array([-0.05400868, -0.07047941, -0.10579343, -0.08200902, -0.0219225 ,
       -0.00469973,  0.00196204, -0.00569309, -0.00681203, -0.00428058,
        0.00437502, -0.0191124 ])

In [19]:
no3_quota = -no3/all_components
no3_quota

array([0.25534668, 0.35926814, 0.37582072, 0.25124891, 0.06831768,
       0.00655672, 0.00098253, 0.0007737 , 0.00177815, 0.00412344,
       0.03451338, 0.11918447])

In [20]:
so4_quota = -so4*2/all_components
so4_quota

array([0.6152849 , 0.47569348, 0.4704718 , 0.60761504, 0.8077361 ,
       0.87661405, 0.8876181 , 0.90164173, 0.90969631, 0.9178901 ,
       0.86391291, 0.7664708 ])

In [21]:
nh4_quota+no2_quota+no3_quota+po4_quota+so4_quota

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

#### Alkalinity to reduced sulfur compounds fluxes fractions

In [22]:
ds1 = xr.open_dataset('data/low_sulfate_reduction_rate/2_po75-25_di1e-9/sediments.nc')
ds2 = xr.open_dataset('data/low_sulfate_reduction_rate/5_po75-25_di10e-9/sediments.nc')
ds3 = xr.open_dataset('data/low_sulfate_reduction_rate/10_po75-25_di35e-9/sediments.nc')

In [23]:
ds = ds2

alkflux_df = ds['B_C_Alk   _flux'].to_dataframe()
nh4flux_df = ds['B_NUT_NH4 _flux'].to_dataframe()
no2flux_df = ds['B_NUT_NO2 _flux'].to_dataframe()
no3flux_df = ds['B_NUT_NO3 _flux'].to_dataframe()
po4flux_df = ds['B_NUT_PO4 _flux'].to_dataframe()
so4flux_df = ds['B_S_SO4   _flux'].to_dataframe()
h2s__flux_df = ds['B_S_H2S   _flux'].to_dataframe()
s0___flux_df = ds['B_S_S0    _flux'].to_dataframe()
s2o3_flux_df = ds['B_S_S2O3  _flux'].to_dataframe()

alkflux_bottom = alkflux_df.groupby('z_faces').get_group(2.5)
nh4flux_bottom = nh4flux_df.groupby('z_faces').get_group(2.5)
no2flux_bottom = no2flux_df.groupby('z_faces').get_group(2.5)
no3flux_bottom = no3flux_df.groupby('z_faces').get_group(2.5)
po4flux_bottom = po4flux_df.groupby('z_faces').get_group(2.5)
so4flux_bottom = so4flux_df.groupby('z_faces').get_group(2.5)
h2sflux_bottom  = h2s__flux_df.groupby('z_faces').get_group(2.5)
s0flux_bottom   = s0___flux_df.groupby('z_faces').get_group(2.5)
s2o3flux_bottom = s2o3_flux_df.groupby('z_faces').get_group(2.5)

In [24]:
alkflux_df = alkflux_df.reset_index()
alkflux_df.z_faces[38]

2.5199999809265137

In [25]:
year = (('2011-01-01','2011-01-31'), ('2011-02-01','2011-02-28'), ('2011-03-01','2011-03-31'), ('2011-04-01','2011-04-30'), 
        ('2011-05-01','2011-05-31'), ('2011-06-01','2011-06-30'), ('2011-07-01','2011-07-31'), ('2011-08-01','2011-08-31'),
        ('2011-09-01','2011-09-30'), ('2011-10-01','2011-10-31'), ('2011-11-01','2011-11-30'), ('2011-12-01','2011-12-31'))

In [26]:
alk_year = []
nh4_year = []
no2_year = []
no3_year = []
po4_year = []
so4_year = []
h2s_year = []
s0_year = []
s2o3_year = []
for month in year:
    alk_month = alkflux_bottom.loc[month[0]:month[1]]
    nh4_month = nh4flux_bottom.loc[month[0]:month[1]]
    no2_month = no2flux_bottom.loc[month[0]:month[1]]
    no3_month = no3flux_bottom.loc[month[0]:month[1]]
    po4_month = po4flux_bottom.loc[month[0]:month[1]]
    so4_month = so4flux_bottom.loc[month[0]:month[1]]
    h2s_month = h2sflux_bottom.loc[month[0]:month[1]]
    s0_month = s0flux_bottom.loc[month[0]:month[1]]
    s2o3_month = s2o3flux_bottom.loc[month[0]:month[1]]
    alk_year.append(alk_month['B_C_Alk   _flux'].mean())
    nh4_year.append(nh4_month['B_NUT_NH4 _flux'].mean())
    no2_year.append(no2_month['B_NUT_NO2 _flux'].mean())
    no3_year.append(no3_month['B_NUT_NO3 _flux'].mean())
    po4_year.append(po4_month['B_NUT_PO4 _flux'].mean())
    so4_year.append(so4_month['B_S_SO4   _flux'].mean())
    h2s_year.append(h2s_month['B_S_H2S   _flux'].mean())
    s0_year.append(s0_month['B_S_S0    _flux'].mean())
    s2o3_year.append(s2o3_month['B_S_S2O3  _flux'].mean())

In [27]:
alk = np.array(alk_year)
nh4 = np.array(nh4_year)
no2 = np.array(no2_year)
no3 = np.array(no3_year)
po4 = np.array(po4_year)
so4 = np.array(so4_year)
h2s = np.array(h2s_year)
s0 = np.array(s0_year)
s2o3 = np.array(s2o3_year)

In [28]:
s_total = h2s + s0 + 2*s2o3

In [29]:
so4direct_quota = -2*so4

In [30]:
so4direct_quota

array([ 1.30717671,  0.79401398,  0.67029887,  1.1521666 ,  3.35178661,
        7.92447186, 10.8350687 ,  9.7854023 ,  7.42817974,  6.08274937,
        5.43908501,  3.14279342])

In [31]:
sulfurquota = s_total*2

In [32]:
sulfurquota

array([2.03801749, 1.17398097, 0.81777972, 0.8962803 , 2.20720802,
       5.68820846, 8.71311671, 7.65851259, 6.15646052, 5.37332326,
       4.92504215, 3.70756191])

In [33]:
sum(so4direct_quota)-sum(sulfurquota)

8.557701054029167

#### Calculation of the average alkalinity values

In [2]:
ls data/low_sulfate_reduction_rate/

10_po75-25_di35e-9/  3_po75-25_di2e-9/   6_po75-25_di15e-9/  9_po75-25_di30e-9/
1_po75-25_di0e-9/    4_po75-25_di5e-9/   7_po75-25_di20e-9/
2_po75-25_di1e-9/    5_po75-25_di10e-9/  8_po75-25_di25e-9/


In [3]:
base_path = 'data/low_sulfate_reduction_rate'

In [4]:
ds1 = xr.open_dataset('{}/1_po75-25_di0e-9/water.nc'.format(base_path))
ds2 = xr.open_dataset('{}/2_po75-25_di1e-9/water.nc'.format(base_path))
ds3 = xr.open_dataset('{}/3_po75-25_di2e-9/water.nc'.format(base_path))
ds4 = xr.open_dataset('{}/4_po75-25_di5e-9/water.nc'.format(base_path))
ds5 = xr.open_dataset('{}/5_po75-25_di10e-9/water.nc'.format(base_path))
ds6 = xr.open_dataset('{}/6_po75-25_di15e-9/water.nc'.format(base_path))
ds7 = xr.open_dataset('{}/7_po75-25_di20e-9/water.nc'.format(base_path))
ds8 = xr.open_dataset('{}/8_po75-25_di25e-9/water.nc'.format(base_path))
ds9 = xr.open_dataset('{}/9_po75-25_di30e-9/water.nc'.format(base_path))
ds10 = xr.open_dataset('{}/10_po75-25_di35e-9/water.nc'.format(base_path))

In [45]:
ds = ds10

In [46]:
alk_df     = ds['B_C_Alk'].to_dataframe()
alk_surface    = alk_df.groupby('z').get_group(0.625)
alk_surface_year = alk_surface.loc['2011-01-01':'2011-12-31']

In [47]:
alk_surface_year = alk_surface_year.reset_index()

In [48]:
alk_surface_year['B_C_Alk'].mean()

2372.77197265625